In [ ]:
#Install Pakeges
pip install tensorflow

In [ ]:
pip install keras

the second experiment


In [1]:
!pip install tensorflow
!pip install tensorflow-gpu
!pip install nltk
!pip install Flask


     |████████████████████████████████| 394.3MB 38kB/s 


In [2]:
# libraries
import random
from keras.optimizers import SGD
from keras.layers import Dense, Dropout
from keras.models import load_model
from keras.models import Sequential
import numpy as np
import pickle
import json
import nltk
from nltk.stem import WordNetLemmatizer


In [3]:
nltk.download("punkt")
nltk.download("wordnet")


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


True

In [4]:
# init file
words = []
classes = []
documents = []
ignore_words = ["?", "!"]
data_file = open("intents.json").read()
intents = json.loads(data_file)

lemmatizer = WordNetLemmatizer() 

In [5]:
for intent in intents["intents"]:
    for pattern in intent["patterns"]:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent["tag"]))

        # adding classes to our class list
        if intent["tag"] not in classes:
            classes.append(intent["tag"])


In [6]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print(len(documents), "documents")

print(len(classes), "classes", classes)

print(len(words), "unique lemmatized words", words)


pickle.dump(words, open("words.pkl", "wb"))
pickle.dump(classes, open("classes.pkl", "wb"))


142 documents
26 classes [' معلومات عن مقام سيدنا ابراهيم ', 'goodbye', 'greeting', 'options', 'thanks', 'ابعاد برج الساعة', 'السلام', 'المهمة', 'برج الساعة', 'بناء الكعبة', 'ترحيب', 'جبل النور', 'شكراً', 'كيف الحال', 'لايوجد جواب', 'مساحة مسجد التنعيم', 'مساعدة', 'مسجد التنعيم', 'معلومات الكعبة', 'معلومات المسعى', 'معلومات جبل النور', 'موقع برج الساعة', 'موقع مسجد التنعيم', 'موقع مقام سيدنا ابراهيم ', 'وداع', 'وصف مقام سيدنا ابراهيم ']
147 unique lemmatized words ["'s", ',', 'anyone', 'are', 'awesome', 'be', 'bye', 'can', 'chatting', 'could', 'day', 'do', 'for', 'good', 'goodbye', 'hello', 'help', 'helpful', 'helping', 'hi', 'how', 'is', 'later', 'me', 'next', 'nice', 'offered', 'provide', 'see', 'support', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'what', 'you', '؟', 'أخبارك', 'أهلا', 'أين', 'إبراهيم', 'ابراهيم', 'ابرهيم', 'ابعاد', 'احتاج', 'اخباركم', 'ارتفاع', 'استخدمك', 'اشكرك', 'الاستخدام', 'التنعيم', 'الجبل', 'الحال', 'الخير', 'الساعة', 'السلام', 'السلامة', 'الشري

In [7]:
# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:, 0])
train_y = list(training[:, 1])
print("Training data created")


Training data created


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray


In [8]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(64, activation="relu"))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation="softmax"))
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               18944     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 64)                8256      
_________________________________________________________________
dropout_1 (Dropout)          (None, 64)                0         
_________________________________________________________________
dense_2 (Dense)              (None, 26)                1690      
Total params: 28,890
Trainable params: 28,890
Non-trainable params: 0
_________________________________________________________________


In [9]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss="categorical_crossentropy", optimizer=sgd, metrics=["accuracy"])



In [10]:
from keras import callbacks 
earlystopping = callbacks.EarlyStopping(monitor ="loss", mode ="min", patience = 5, restore_best_weights = True)
callbacks =[earlystopping]


In [11]:
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save("chatbot_model.h5", hist)
print("model created")


Epoch 1/200
29/29 [==============================] - 1s 1ms/step - loss: 3.2697 - accuracy: 0.0645 
Epoch 2/200
29/29 [==============================] - 0s 1ms/step - loss: 3.2183 - accuracy: 0.0907
Epoch 3/200
29/29 [==============================] - 0s 1ms/step - loss: 3.1117 - accuracy: 0.1670
Epoch 4/200
29/29 [==============================] - 0s 1ms/step - loss: 2.8766 - accuracy: 0.1966
Epoch 5/200
29/29 [==============================] - 0s 1ms/step - loss: 2.5954 - accuracy: 0.2982
Epoch 6/200
29/29 [==============================] - 0s 1ms/step - loss: 2.4675 - accuracy: 0.3395
Epoch 7/200
29/29 [==============================] - 0s 1ms/step - loss: 2.3583 - accuracy: 0.3120
Epoch 8/200
29/29 [==============================] - 0s 1ms/step - loss: 2.2689 - accuracy: 0.3734
Epoch 9/200
29/29 [==============================] - 0s 1ms/step - loss: 2.0717 - accuracy: 0.3896
Epoch 10/200
29/29 [==============================] - 0s 1ms/step - loss: 1.8603 - accuracy: 0.4899
Epoch 11

In [12]:
!pip install flask
!pip install flask-ngrok

In [13]:
# libraries
import random
import numpy as np
import pickle
import json
from flask import Flask, render_template, request
from flask_ngrok import run_with_ngrok
import nltk
from keras.models import load_model
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()


# chat initialization
model = load_model("chatbot_model.h5")
intents = json.loads(open("intents.json").read())
words = pickle.load(open("words.pkl", "rb"))
classes = pickle.load(open("classes.pkl", "rb"))

app = Flask(__name__, template_folder='/content')
run_with_ngrok(app) 

@app.route("/")
def home():
    return render_template("index.html")


In [14]:
@app.route("/get", methods=["POST"])
def chatbot_response():
    msg = request.form["msg"]
    if msg.startswith('my name is'):
        name = msg[11:]
        ints = predict_class(msg, model)
        res1 = getResponse(ints, intents)
        res =res1.replace("{n}",name)
    elif msg.startswith('hi my name is'):
        name = msg[14:]
        ints = predict_class(msg, model)
        res1 = getResponse(ints, intents)
        res =res1.replace("{n}",name)
    else:
        ints = predict_class(msg, model)
        res = getResponse(ints, intents)
    return res


In [15]:
def clean_up_sentence(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    sentence_words = [lemmatizer.lemmatize(word.lower()) for word in sentence_words]
    return sentence_words


# return bag of words array: 0 or 1 for each word in the bag that exists in the sentence
def bow(sentence, words, show_details=True):
    # tokenize the pattern
    sentence_words = clean_up_sentence(sentence)
    # bag of words - matrix of N words, vocabulary matrix
    bag = [0] * len(words)
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                # assign 1 if current word is in the vocabulary position
                bag[i] = 1
                if show_details:
                    print("found in bag: %s" % w)
    return np.array(bag)


In [16]:
def predict_class(sentence, model):
    # filter out predictions below a threshold
    p = bow(sentence, words, show_details=False)
    res = model.predict(np.array([p]))[0]
    ERROR_THRESHOLD = 0.25
    results = [[i, r] for i, r in enumerate(res) if r > ERROR_THRESHOLD]
    # sort by strength of probability
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = []
    for r in results:
        return_list.append({"intent": classes[r[0]], "probability": str(r[1])})
    return return_list


def getResponse(ints, intents_json):
    tag = ints[0]["intent"]
    list_of_intents = intents_json["intents"]
    for i in list_of_intents:
        if i["tag"] == tag:
            result = random.choice(i["responses"])
            break
    return result



if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)


 * Running on http://78094f3633a7.ngrok.io
 * Traffic stats available on http://127.0.0.1:4040
